# Build an Interactive Dashboard with Ploty Dash

### Dash Installation

In [23]:
pip install jupyter-dash

Note: you may need to restart the kernel to use updated packages.


In [24]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update


In [5]:
df=pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv')

df.head(10)

,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,Success,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,Success,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,Success,0,677.0,F9 v1.0 B0007,v1.0
5,5,7,CCAFS LC-40,Success,0,3170.0,F9 v1.1,v1.1
6,6,8,CCAFS LC-40,Success,0,3325.0,F9 v1.1,v1.1
7,7,9,CCAFS LC-40,Success,0,2296.0,F9 v1.1,v1.1
8,8,10,CCAFS LC-40,Success,0,1316.0,F9 v1.1,v1.1
9,9,11,CCAFS LC-40,Success,0,4535.0,F9 v1.1,v1.1


In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

launch_sites1 = spacex_df['Launch Site'].unique().tolist()
launch_sites = []
launch_sites.append({'label': 'All Sites', 'value': 'All Sites'})
for launch_site in launch_sites1:
    launch_sites.append({'label': launch_site, 'value': launch_site})
    
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[
                            html.Div([
                            html.H1('SpaceX Launch Records Dashboard',
                            style={'textAlign': 'center', 'color': '#503D36','font-size': 40}),
                                    ]),
                    html.Div([
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                            dcc.Dropdown(
                                    id = 'sitesdropdown',
                                    options = [{'label': 'All Sites', 'value': 'ALL'},
                                                {'label': 'CCAFS SLC 40', 'label': 'CCAFS SLC 40'},
                                                {'label':'KSC LC 39A','value':'KSC LC 39A'},
                                                {'label':'VAFB SLC 4E','value':'VAFB SLC 4E'}],            
                                    placeholder = 'Select a Launch Site here',
                                    searchable = True 
                            ),


                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                            html.Div(dcc.Graph(
                                    id='success-pie-chart')),
                            ],
                    html.Div([
                                html.Div("Payload range (Kg):"),
                                html.Div([
                                        # TASK 3: Add a slider to select payload range
                                        dcc.RangeSlider(id='id',
                                                min=0, max=10000, step=1000,
                                                marks = {
                                                 0: {'label': '0 Kg'},
                                                1000: {'label': '1000 Kg'},
                                                2000: {'label': '2000 Kg'},
                                                3000: {'label': '3000 Kg'},
                                                4000: {'label': '4000 Kg'},
                                                5000: {'label': '5000 Kg'},
                                                6000: {'label': '6000 Kg'},
                                                7000: {'label': '7000 Kg'},
                                                8000: {'label': '8000 Kg'},
                                                9000: {'label': '9000 Kg'},
                                                10000: {'label': '10000 Kg'}},
                                                value=[min_payload, max_payload]
                                                         ),
                                          ],                                    
                                
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                     ]),
                    ]

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(ss):
    filtered_df = spacex_df
    if (ss == 'All Sites':
        data = spacex_df[spacex_df['class'] == 1],
        fig = px.pie(data, 
                    names = 'Launch Site',
                    title = 'Total Success Launches by All Sites',
                    hole = .2,
                    color_discrete_sequence = px.colors.sequential.RdBu
                    )
    else:
        site_specific  = spacex_df.loc[spacex_df['Launch Site'] == ss]
        fig = px.pie(
                site_specific,
                names = 'Success Rate',
                title = 'Total Success Launches for Site &#8608; '+ss
                hole = .2
                            )
        return fig
        # return the outcomes piechart for a selected site
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
     Output(component_id = 'success-payload-scatter-chart', component_property = 'figure'),
     [Input(component_id = 'site-dropdown', component_property = 'value'), 
     Input(component_id = "payload_slider", component_property = "value")]
)
def update_scattergraph(ss,payload_slider):
    if (ss== 'All Sites'):
        low, high = payload_slider
        inrange = (spacex_df['Payload Mass (kg)'] > low) & (spacex_df['Payload Mass (kg)'] < high)
        fig = px.scatter(
                spacex_df[inrange], 
                x = "Payload Mass (kg)", 
                y = "class",
                title = 'Correlation Between Payload and Success for All Sites',
                color="Booster Version Category",
                size='Payload Mass (kg)',
                hover_data=['Payload Mass (kg)']
            )
    else:
        low, high = payload_slider
        site_specific  = spacex_df.loc[spacex_df['Launch Site'] == sitesdropdown]
        inrange = (sitesdropdown['Payload Mass (kg)'] > low) & (site_specific['Payload Mass (kg)'] < high)
        fig = px.scatter(
                site_specific[inrange],
                x = "Payload Mass (kg)",
                y = "class",
                title = 'Correlation Between Payload and Success for Site &#8608; '+sitesdropdown,
                color="Booster Version Category",
                size='Payload Mass (kg)',
                hover_data=['Payload Mass (kg)']
            )
    return fig


# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
